### Compute desired orderings

In [432]:
import random
random.seed(1)
import sys
sys.path.append("../../src/")
import uncertainpy.gradual as grad
import numpy as np
from tqdm import tqdm
import time

In [433]:
# obtain a QBAF and set the gradual semantics

# Computation via Forward Propagation
bag = grad.BAG("../../bags/mlp_0.bag")

# # QE
# agg_f = grad.semantics.modular.SumAggregation()
# inf_f = grad.semantics.modular.QuadraticMaximumInfluence(conservativeness=1)

# # DF-QuAD
# agg_f = grad.semantics.modular.ProductAggregation()
# inf_f = grad.semantics.modular.LinearInfluence(conservativeness=1)

# # SD-DF-QUAD
# agg_f = grad.semantics.modular.ProductAggregation()
# inf_f = grad.semantics.modular.SQDFQUADInfluence(conservativeness=1)

# EB
agg_f = grad.semantics.modular.SumAggregation()
inf_f = grad.semantics.modular.EulerBasedInfluence()

# # EBT
# agg_f = grad.semantics.modular.TopAggregation()
# inf_f = grad.semantics.modular.EulerBasedInfluence()

#returns dictionary of strength values if needed
strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

for arg in bag.arguments.values():
    print((arg.name,arg.strength))

('0', 0.3799999999999999)
('1', 0.19999999999999996)
('2', 0.0)
('3', 0.28)
('4', 0.6000000000000001)
('5', 0.88)
('6', 0.83)
('7', 0.51)
('8', 0.9931293674323816)
('9', 0.6974160343046287)
('10', 0.8709189591317626)
('11', 0.6797230199692126)
('12', 0.8172379290330771)
('13', 0.9894039099374837)
('14', 0.2941036449456075)
('15', 0.14344438669621074)
('16', 0.413193629825814)
('17', 0.44513470637319275)
('18', 0.589320748600175)
('19', 0.0)
('20', 0.47105208124257303)
('21', 0.19984399200927905)
('22', 0.8384354496730594)
('23', 0.7522659588988032)
('24', 0.49051156539443475)
('25', 0.8208204549430622)
('26', 0.9489788877298626)
('27', 0.671042407950527)
('28', 0.3908487258255946)
('29', 0.5898714219154544)
('30', 0.5300832938770527)
('31', 0.5700240665590468)
('32', 0.7762843021509616)
('33', 0.2887315737662993)
('34', 0.4295703332896532)
('35', 0.7016808989284933)
('36', 0.8888481740710188)
('37', 0.7447624338111591)
('38', 0.9015794195235514)
('39', 0.9600338342685802)


In [434]:
layer_sizes = [8,8,8,8,8]
N = 100

In [435]:
def get_layer_nodes(layer_sizes, layer_index=None):
    node_id = 0
    for i, size in enumerate(layer_sizes):
        if i == layer_index:
            return [str(n) for n in range(node_id, node_id + size)]
        node_id += size

    raise ValueError(f"Invalid layer_index {layer_index}: must be between 0 and {len(layer_sizes) - 1}")

In [436]:
preferred_order = get_layer_nodes(layer_sizes, len(layer_sizes)-1)
preferred_order

['32', '33', '34', '35', '36', '37', '38', '39']

In [437]:
preferred_strengths = np.linspace(1, 0, len(preferred_order)).tolist() # for the purpose of comparing kendall
preferred_strengths

[1.0,
 0.8571428571428572,
 0.7142857142857143,
 0.5714285714285714,
 0.4285714285714286,
 0.2857142857142858,
 0.1428571428571429,
 0.0]

In [438]:
immutable_args = get_layer_nodes(layer_sizes, len(layer_sizes)-2)
immutable_args

['24', '25', '26', '27', '28', '29', '30', '31']

In [439]:
# # simple ReLU loss function (linear)
# def compute_loss(bag, preferred_order):
#     loss = 0
#     for i in range(len(preferred_order) - 1):
#         loss += max(0, (bag.arguments[preferred_order[i+1]].strength - bag.arguments[preferred_order[i]].strength))
#     # print(f"Loss: {loss}")
#     return loss

In [440]:
# pairwise smooth logistic loss (quadratic)
def compute_loss(bag, preferred_order,delta=0):
    loss = 0
    # traverse all the pairs in preferred_order, i < j
    for i in range(len(preferred_order) - 1):
        for j in range(i + 1, len(preferred_order)):
            A_i = preferred_order[i]
            A_j = preferred_order[j]
            # obtain strength σ(A_i) 和 σ(A_j)
            sigma_i = bag.arguments[A_i].strength
            sigma_j = bag.arguments[A_j].strength
            loss += np.log(1 + np.exp(sigma_j - sigma_i + delta))
    return loss

In [441]:
# compute gradient for the loss function
def compute_gradient(h, bag, preferred_order):
    # h is the perturbation score
    gradient = {}
    penalty = compute_loss(bag, preferred_order)
    for arg in bag.arguments.values():

        initial_weight = arg.get_initial_weight() # record initial base score
        arg.reset_initial_weight(initial_weight+h) # perturb base scores
        grad.algorithms.computeStrengthValues(bag, agg_f, inf_f) # recompute strength with new base score
        # for arg in bag.arguments.values():
        #     print((arg.name,arg.strength))

        new_penalty = compute_loss(bag, preferred_order)
        # print(new_penalty)

        gradient[arg.name] = (new_penalty-penalty)/h # compute gradient
        # print(f"gradient for argument {arg.name} is {gradient[arg.name]}")

        arg.reset_initial_weight(initial_weight) # set it back to the original base score after computing gradient
        grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)
        # for arg in bag.arguments.values():
        #     print((arg.name,arg.strength))

    return gradient

In [442]:
h = 10e-6
compute_gradient(h, bag, preferred_order)

{'0': 0.0077717015045664075,
 '1': -0.0026040751777145488,
 '2': -0.0022694724322036564,
 '3': 0.003474086796018127,
 '4': -0.003174797313931776,
 '5': -0.010935207939155587,
 '6': 0.007223679432399876,
 '7': 0.006722783396639897,
 '8': -0.029334794504620728,
 '9': -0.011687119894077112,
 '10': 0.019133939233029196,
 '11': 0.010557868534988302,
 '12': 0.015230538963351135,
 '13': -0.021686563300704616,
 '14': 0.011876456795789634,
 '15': 0.03240940280591076,
 '16': -0.20721334337281402,
 '17': 0.15574882681335112,
 '18': -0.011131823995924604,
 '19': 0.5608460654116243,
 '20': -0.1164027214173302,
 '21': -0.06469481164117497,
 '22': -0.01585864772835066,
 '23': -0.06230306262011708,
 '24': -0.06719551137734925,
 '25': 0.07333528557751379,
 '26': -0.3543669986783015,
 '27': -0.17193586643315936,
 '28': 0.18490375417457014,
 '29': 0.8821868643593688,
 '30': 0.13402047898125602,
 '31': 1.125280734726175,
 '32': -2.3301771584272046,
 '33': -3.1672148182337874,
 '34': -2.54949380718017,
 '3

In [443]:
# adam optimiser
def adam_gradient(name, gradient, m, v, i):

    grad = gradient[name]
    # Adam optimiser parameters
    learning_rate = 0.01  # Initial learning rate
    beta1 = 0.9            # First-order moment decay rate
    beta2 = 0.999          # Second-order moment decay rate
    epsilon = 1e-8         # a small constant

    # update first-order moment and second-order moment
    m = beta1 * m + (1 - beta1) * grad
    v = beta2 * v + (1 - beta2) * (grad ** 2)

    # bias correction
    m_hat = m / (1 - beta1 ** i)
    v_hat = v / (1 - beta2 ** i)

    update = learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

    return update, m, v

In [444]:
# if the order is exactly the same as the desired order, then valid otherwise not.
def is_valid_order(bag, preferred_order):
    strengths = [bag.arguments[name].strength for name in preferred_order]
    # print(strengths)
    return all(strengths[i] >= strengths[i+1] for i in range(len(strengths)-1))
print(is_valid_order(bag, preferred_order))

False


In [445]:
from scipy.stats import kendalltau
# preferred_strengths = [0.9, 0.60, 0.59] the preferred strength values
# predicted_strengths = [0.85, 0.80, 0.79] the computed strength values
def compute_kendall(preferred_strengths,predicted_strengths):
    tau, _ = kendalltau(preferred_strengths, predicted_strengths)
    return tau
# compute_kendall(preferred_strengths,predicted_strengths)

In [446]:
# test the validity for N MLP-like QBAFs

time_total = [0] * N
valid = [0] * N
kendall = [0] * N
for i in tqdm(range(N)):
    start = time.time()
    filename = f'../../bags/mlp_{i}.bag'
    bag = grad.BAG(filename)
    m = {}
    v = {}
    for epoch in range(1, 1001):


        # print(f"Epoch:{epoch}")
        # compute gradient for all arguments
        gradient = compute_gradient(10e-6, bag, preferred_order)
        # print(f"gradient:{gradient}")
        if all(value == 0 for value in gradient.values()): break

        # update Adam state and update base scores
        for arg in bag.arguments.values():
            if arg.name not in immutable_args:
                if arg.name not in m:
                    m[arg.name] = 0
                    v[arg.name] = 0


                current_weight = arg.get_initial_weight()
                adam_update, m[arg.name], v[arg.name] = adam_gradient(arg.name, gradient, m[arg.name], v[arg.name], epoch)
                new_weight = current_weight - adam_update
                new_weight = max(0, min(1, new_weight))
                arg.reset_initial_weight(new_weight)


        # recompute the strength and penalty
        grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)
        if is_valid_order(bag, preferred_order): break
    if is_valid_order(bag, preferred_order):
        valid[i] = 1
    predicted_strengths = [bag.arguments[name].strength for name in preferred_order]
    kendall[i] = compute_kendall(preferred_strengths,predicted_strengths)
    end = time.time()
    time_total[i] = end-start


print(f"valid:{valid}")
print(f"kendall:{kendall}")
print(f"valid_avg:{sum(valid)/N}")
print(f"kendall_avg:{sum(kendall)/N}")
print(f"runtime:{sum(time_total)/N}")

100%|██████████| 100/100 [35:22<00:00, 21.23s/it]

valid:[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
kendall:[-0.6428571428571428, 0.4999999999999999, 0.14285714285714285, 0.0, -0.3571428571428571, 0.0, -0.21428571428571427, 0.07142857142857142, 0.07142857142857142, 0.21428571428571427, -0.21428571428571427, 0.7142857142857142, 0.21428571428571427, 0.07142857142857142, 0.21428571428571427, -0.07142857142857142, 0.2857142857142857, 0.07142857142857142, -0.2857142857142857, 0.3571428571428571, 0.2857142857142857, 0.3571428571428571, 0.2857142857142857, -0.42857142857142855, 0.3571428571428571, 0.07142857142857142, -0.21428571428571427, -0.07142857142857142, 0.2857142857142857, -0.07142857142857142, -0.3571428571428571, 0.21428571428571427, -0.3571428571428571, 0.6428571428571428, 0.142

In [447]:
# for i in range(len(valid)):
#     if valid[i]==0:
#         print(i)